# TP sur la Théorie des Tests Statistiques

Dans ce TP, nous allons explorer les tests paraméétriques sur des lois normales, en particulier :
1. Tests sur la moyenne d'une loi normale (avec variance connue ou inconnue)
2. Tests sur la variance d'une loi normale (avec moyenne connue)

Ces tests sont fondamentaux en statistique inférentielle et constituent la base de nombreuses méthodes d'analyse statistique.

In [ ]:
# Importation des bibliothèques nécessaires
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm, t


# Configuration pour de meilleurs graphiques
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 12

## 1. Tests sur la moyenne d'une loi normale

Considérons l'hypothèse suivante :

On suppose que l'on dispose de $(X_1, X_2, \ldots, X_n)$ un échantillon issu de $X$ qui suit une loi $\mathcal{N}(\mu, \sigma^2)$. 

Nous allons examiner les tests pour $H_0 : \mu = \mu_0$ contre différentes alternatives.

### 1.1 Génération d'un échantillon pour la démonstration

In [ ]:
import warnings

# Paramètres pour notre simulation
np.random.seed(0)  # Pour la reproductibilité
n = 400  # Taille de l'échantillon
mu_true = 5  # Vraie moyenne de la population
sigma_true = 2  # Vraie écart-type de la population

# Génération d'un échantillon
sample = np.random.normal(mu_true, sigma_true, n)

# Statistiques descriptives
x_bar = np.mean(sample)
s = np.std(sample, ddof=1)  # ddof=1 pour l'estimateur non biaisé

print(f"Moyenne de l'échantillon: {x_bar:.4f}")
print(f"Écart-type de l'échantillon: {s:.4f}")

# Visualisation de l'échantillon
plt.figure(figsize=(10, 6))
plt.hist(sample, bins=15, alpha=0.7, density=True, label='Échantillon')

# Superposition de la densité théorique
x = np.linspace(min(sample) - 2, max(sample) + 2, 1000)
plt.plot(x, norm.pdf(x, mu_true, sigma_true), 'r-', lw=2, 
         label=f'$\mathcal{{N}}({mu_true}, {sigma_true}^2)$')
plt.plot(x, norm.pdf(x, x_bar, s), 'g--', lw=2, 
         label=f'$\mathcal{{N}}({x_bar:.2f}, {s:.2f}^2)$')

plt.axvline(x=mu_true, color='r', linestyle='-', alpha=0.5, label='$\\mu$ réel')
plt.axvline(x=x_bar, color='g', linestyle='--', alpha=0.5, label='$\\bar{X}$ échantillon')

plt.title("Distribution de l'échantillon vs. Distribution théorique")
plt.xlabel('Valeur')
plt.ylabel('Densité')
plt.legend()
plt.grid(True)


# ignore SyntaxWarning
warnings.filterwarnings("ignore", category=SyntaxWarning, message="invalid escape sequence.*")

plt.show()

### 1.2 Test sur la moyenne avec variance connue

Si $\sigma^2$ est connu, on utilise la fonction pivotale $Z_0 = \frac{\bar{X} - \mu_0}{\sigma/\sqrt{n}} \sim \mathcal{N}(0, 1)$

#### Cas bilatéral: $H_0 : \mu = \mu_0$ contre $H_1 : \mu \neq \mu_0$
- Région critique: $RC_\alpha = \{|Z_0| > z_{\alpha/2}\}$

#### Cas unilatéral à gauche: $H_0 : \mu = \mu_0$ (ou $\mu \geq \mu_0$) contre $H_1 : \mu < \mu_0$
- Région critique: $RC_\alpha = \{Z_0 < -z_{\alpha}\}$

#### Cas unilatéral à droite: $H_0 : \mu = \mu_0$ (ou $\mu \leq \mu_0$) contre $H_1 : \mu > \mu_0$
- Région critique: $RC_\alpha = \{Z_0 > z_{\alpha}\}$

In [ ]:
def test_mean_known_variance(sample, mu0, sigma_known, alpha=0.05, alternative='two-sided'):
    """Test sur la moyenne d'une loi normale avec variance connue.
    
    Parameters:
    -----------
    sample : array-like
        Échantillon à tester
    mu0 : float
        Valeur de la moyenne sous H0
    sigma_known : float
        Écart-type connu de la population
    alpha : float, optional
        Niveau de signification (default=0.05)
    alternative : {'two-sided', 'less', 'greater'}, optional
        Type d'alternative (default='two-sided')
        
    Returns:
    --------
    dict
        Résultats du test (statistique, p-value, décision)
    """
    n = len(sample)
    x_bar = np.mean(sample)
    z0 = (x_bar - mu0) / (sigma_known / np.sqrt(n))
    
    # Calcul de la p-value selon l'alternative
    if alternative == 'two-sided':
        p_value = 2 * norm.sf(abs(z0))
        critical_value = norm.ppf(1 - alpha/2)
        reject = abs(z0) > critical_value
    elif alternative == 'less':
        p_value = norm.cdf(z0)
        critical_value = norm.ppf(alpha)
        reject = z0 < critical_value
    elif alternative == 'greater':
        p_value = norm.sf(z0)
        critical_value = norm.ppf(1 - alpha)
        reject = z0 > critical_value
    else:
        raise ValueError("alternative must be 'two-sided', 'less' or 'greater'")
    
    # Décision
    decision = "Rejeter H0" if reject else "Ne pas rejeter H0"
    
    return {
        'statistic': z0,
        'p_value': p_value,
        'critical_value': critical_value,
        'decision': decision,
        'reject': reject
    }

In [ ]:
# Test avec variance connue
mu0 = 4.5  # Valeur hypothétique de la moyenne
sigma_known = 2  # Variance connue (dans un cas réel, cette valeur serait fournie)
alpha = 0.05  # Niveau de signification

# Test bilatéral
result_bilateral = test_mean_known_variance(sample, mu0, sigma_known, alpha, 'two-sided')
print("\nTest bilatéral H0: μ = μ0 vs H1: μ ≠ μ0")
print(f"Statistique Z0 = {result_bilateral['statistic']:.4f}")
print(f"Valeur critique z{alpha/2} = {result_bilateral['critical_value']:.4f}")
print(f"p-value = {result_bilateral['p_value']:.4f}")
print(f"Décision ({alpha=}): {result_bilateral['decision']}")

# Test unilatéral à gauche
result_left = test_mean_known_variance(sample, mu0, sigma_known, alpha, 'less')
print("\nTest unilatéral à gauche H0: μ = μ0 (ou μ ≥ μ0) vs H1: μ < μ0")
print(f"Statistique Z0 = {result_left['statistic']:.4f}")
print(f"Valeur critique -z{alpha} = {result_left['critical_value']:.4f}")
print(f"p-value = {result_left['p_value']:.4f}")
print(f"Décision ({alpha=}): {result_left['decision']}")

# Test unilatéral à droite
result_right = test_mean_known_variance(sample, mu0, sigma_known, alpha, 'greater')
print("\nTest unilatéral à droite H0: μ = μ0 (ou μ ≤ μ0) vs H1: μ > μ0")
print(f"Statistique Z0 = {result_right['statistic']:.4f}")
print(f"Valeur critique z{alpha} = {result_right['critical_value']:.4f}")
print(f"p-value = {result_right['p_value']:.4f}")
print(f"Décision ({alpha=}): {result_right['decision']}")

### Visualisation du test (variance connue)

In [ ]:
def plot_test_normal(result, test_type='two-sided', alpha=0.05):
    """Visualise la région critique et la statistique de test pour une loi normale.
    
    Parameters:
    -----------
    result : dict
        Résultats du test
    test_type : {'two-sided', 'less', 'greater'}, optional
        Type de test (default='two-sided')
    alpha : float, optional
        Niveau de signification (default=0.05)
    """
    z0 = result['statistic']
    z = np.linspace(-4, 4, 1000)
    pdf_z = norm.pdf(z)
    
    plt.figure(figsize=(12, 6))
    plt.plot(z, pdf_z, 'b-', lw=2, label='$\mathcal{N}(0, 1)$')
    
    if test_type == 'two-sided':
        # Région critique bilatérale
        z_crit = norm.ppf(1 - alpha/2)
        idx_left = z <= -z_crit
        idx_right = z >= z_crit
        plt.fill_between(z[idx_left], 0, pdf_z[idx_left], color='r', alpha=0.5, label=f'RC (α={alpha})')
        plt.fill_between(z[idx_right], 0, pdf_z[idx_right], color='r', alpha=0.5)
        plt.axvline(x=-z_crit, color='r', linestyle='--', alpha=0.7, label=f'-z_{{α/2}}={-z_crit:.4f}')
        plt.axvline(x=z_crit, color='r', linestyle='--', alpha=0.7, label=f'z_{{α/2}}={z_crit:.4f}')
        plt.title(f"Test bilatéral sur la moyenne (H0: μ = μ0 vs H1: μ ≠ μ0)")
    
    elif test_type == 'less':
        # Région critique unilatérale à gauche
        z_crit = norm.ppf(alpha)
        idx = z <= z_crit
        plt.fill_between(z[idx], 0, pdf_z[idx], color='r', alpha=0.5, label=f'RC (α={alpha})')
        plt.axvline(x=z_crit, color='r', linestyle='--', alpha=0.7, label=f'-z_{{α}}={z_crit:.4f}')
        plt.title(f"Test unilatéral à gauche (H0: μ = μ0 ou μ ≥ μ0 vs H1: μ < μ0)")
    
    elif test_type == 'greater':
        # Région critique unilatérale à droite
        z_crit = norm.ppf(1 - alpha)
        idx = z >= z_crit
        plt.fill_between(z[idx], 0, pdf_z[idx], color='r', alpha=0.5, label=f'RC (α={alpha})')
        plt.axvline(x=z_crit, color='r', linestyle='--', alpha=0.7, label=f'z_{{α}}={z_crit:.4f}')
        plt.title(f"Test unilatéral à droite (H0: μ = μ0 ou μ ≤ μ0 vs H1: μ > μ0)")
    
    # Statistique calculée
    plt.axvline(x=z0, color='g', lw=2, label=f'Z0={z0:.4f}')
    
    # Décision
    if result['reject']:
        plt.text(0.05, 0.95, "Décision: Rejeter H0", transform=plt.gca().transAxes, 
                 bbox=dict(facecolor='red', alpha=0.1))
    else:
        plt.text(0.05, 0.95, "Décision: Ne pas rejeter H0", transform=plt.gca().transAxes, 
                 bbox=dict(facecolor='green', alpha=0.1))
    
    plt.xlabel('z')
    plt.ylabel('Densité')
    plt.legend()
    plt.grid(True)
    plt.show()

In [ ]:
# Visualisation des tests pour variance connue
plot_test_normal(result_bilateral, 'two-sided', alpha)
plot_test_normal(result_left, 'less', alpha)
plot_test_normal(result_right, 'greater', alpha)

### 1.3 Test sur la moyenne avec variance inconnue

Si $\sigma^2$ est inconnu, on utilise la fonction pivotale $T_0 = \frac{\bar{X} - \mu_0}{S/\sqrt{n}} \sim T(n-1)$

#### Cas bilatéral: $H_0 : \mu = \mu_0$ contre $H_1 : \mu \neq \mu_0$
- Région critique: $RC_\alpha = \{|T_0| > t_{n-1,\alpha/2}\}$

#### Cas unilatéral à gauche: $H_0 : \mu = \mu_0$ (ou $\mu \geq \mu_0$) contre $H_1 : \mu < \mu_0$
- Région critique: $RC_\alpha = \{T_0 < -t_{n-1,\alpha}\}$

#### Cas unilatéral à droite: $H_0 : \mu = \mu_0$ (ou $\mu \leq \mu_0$) contre $H_1 : \mu > \mu_0$
- Région critique: $RC_\alpha = \{T_0 > t_{n-1,\alpha}\}$

In [ ]:
def test_mean_unknown_variance(sample, mu0, alpha=0.05, alternative='two-sided'):
    """Test sur la moyenne d'une loi normale avec variance inconnue.
    
    Parameters:
    -----------
    sample : array-like
        Échantillon à tester
    mu0 : float
        Valeur de la moyenne sous H0
    alpha : float, optional
        Niveau de signification (default=0.05)
    alternative : {'two-sided', 'less', 'greater'}, optional
        Type d'alternative (default='two-sided')
        
    Returns:
    --------
    dict
        Résultats du test (statistique, p-value, décision)
    """
    n = len(sample)
    x_bar = np.mean(sample)
    s = np.std(sample, ddof=1)  # ddof=1 pour l'estimateur non biaisé
    t0 = (x_bar - mu0) / (s / np.sqrt(n))
    
    # Degrés de liberté
    df = n - 1
    
    # Calcul de la p-value selon l'alternative
    if alternative == 'two-sided':
        p_value = 2 * t.sf(abs(t0), df)
        critical_value = t.ppf(1 - alpha/2, df)
        reject = abs(t0) > critical_value
    elif alternative == 'less':
        p_value = t.cdf(t0, df)
        critical_value = t.ppf(alpha, df)
        reject = t0 < critical_value
    elif alternative == 'greater':
        p_value = t.sf(t0, df)
        critical_value = t.ppf(1 - alpha, df)
        reject = t0 > critical_value
    else:
        raise ValueError("alternative must be 'two-sided', 'less' or 'greater'")
    
    # Décision
    decision = "Rejeter H0" if reject else "Ne pas rejeter H0"
    
    return {
        'statistic': t0,
        'p_value': p_value,
        'critical_value': critical_value,
        'decision': decision,
        'reject': reject,
        'df': df
    }

In [ ]:
# Test avec variance inconnue
mu0 = 4.5  # Valeur hypothétique de la moyenne
alpha = 0.05  # Niveau de signification

# Test bilatéral
result_bilateral_t = test_mean_unknown_variance(sample, mu0, alpha, 'two-sided')
print("\nTest bilatéral H0: μ = μ0 vs H1: μ ≠ μ0")
print(f"Statistique T0 = {result_bilateral_t['statistic']:.4f}")
print(f"Degrés de liberté = {result_bilateral_t['df']}")
print(f"Valeur critique t_{result_bilateral_t['df']},{alpha/2} = {result_bilateral_t['critical_value']:.4f}")
print(f"p-value = {result_bilateral_t['p_value']:.4f}")
print(f"Décision ({alpha=}): {result_bilateral_t['decision']}")

# Test unilatéral à gauche
result_left_t = test_mean_unknown_variance(sample, mu0, alpha, 'less')
print("\nTest unilatéral à gauche H0: μ = μ0 (ou μ ≥ μ0) vs H1: μ < μ0")
print(f"Statistique T0 = {result_left_t['statistic']:.4f}")
print(f"Valeur critique -t_{result_left_t['df']},{alpha} = {result_left_t['critical_value']:.4f}")
print(f"p-value = {result_left_t['p_value']:.4f}")
print(f"Décision ({alpha=}): {result_left_t['decision']}")

# Test unilatéral à droite
result_right_t = test_mean_unknown_variance(sample, mu0, alpha, 'greater')
print("\nTest unilatéral à droite H0: μ = μ0 (ou μ ≤ μ0) vs H1: μ > μ0")
print(f"Statistique T0 = {result_right_t['statistic']:.4f}")
print(f"Valeur critique t_{result_right_t['df']},{alpha} = {result_right_t['critical_value']:.4f}")
print(f"p-value = {result_right_t['p_value']:.4f}")
print(f"Décision ({alpha=}): {result_right_t['decision']}")

### Visualisation du test (variance inconnue)

In [ ]:
def plot_test_student(result, test_type='two-sided', alpha=0.05):
    """Visualise la région critique et la statistique de test pour une loi de Student.
    
    Parameters:
    -----------
    result : dict
        Résultats du test
    test_type : {'two-sided', 'less', 'greater'}, optional
        Type de test (default='two-sided')
    alpha : float, optional
        Niveau de signification (default=0.05)
    """
    t0 = result['statistic']
    df = result['df']
    x = np.linspace(-4, 4, 1000)
    pdf_t = t.pdf(x, df)
    
    plt.figure(figsize=(12, 6))
    plt.plot(x, pdf_t, 'b-', lw=2, label=f'$T({df})$')
    
    if test_type == 'two-sided':
        # Région critique bilatérale
        t_crit = t.ppf(1 - alpha/2, df)
        idx_left = x <= -t_crit
        idx_right = x >= t_crit
        plt.fill_between(x[idx_left], 0, pdf_t[idx_left], color='r', alpha=0.5, label=f'RC (α={alpha})')
        plt.fill_between(x[idx_right], 0, pdf_t[idx_right], color='r', alpha=0.5)
        plt.axvline(x=-t_crit, color='r', linestyle='--', alpha=0.7, label=f'-t_{{{df},α/2}}={-t_crit:.4f}')
        plt.axvline(x=t_crit, color='r', linestyle='--', alpha=0.7, label=f't_{{{df},α/2}}={t_crit:.4f}')
        plt.title(f"Test bilatéral sur la moyenne avec variance inconnue (H0: μ = μ0 vs H1: μ ≠ μ0)")
    
    elif test_type == 'less':
        # Région critique unilatérale à gauche
        t_crit = t.ppf(alpha, df)
        idx = x <= t_crit
        plt.fill_between(x[idx], 0, pdf_t[idx], color='r', alpha=0.5, label=f'RC (α={alpha})')
        plt.axvline(x=t_crit, color='r', linestyle='--', alpha=0.7, label=f'-t_{{{df},α}}={t_crit:.4f}')
        plt.title(f"Test unilatéral à gauche avec variance inconnue (H0: μ = μ0 ou μ ≥ μ0 vs H1: μ < μ0)")
    
    elif test_type == 'greater':
        # Région critique unilatérale à droite
        t_crit = t.ppf(1 - alpha, df)
        idx = x >= t_crit
        plt.fill_between(x[idx], 0, pdf_t[idx], color='r', alpha=0.5, label=f'RC (α={alpha})')
        plt.axvline(x=t_crit, color='r', linestyle='--', alpha=0.7, label=f't_{{{df},α}}={t_crit:.4f}')
        plt.title(f"Test unilatéral à droite avec variance inconnue (H0: μ = μ0 ou μ ≤ μ0 vs H1: μ > μ0)")
    
    # Statistique calculée
    plt.axvline(x=t0, color='g', lw=2, label=f'T0={t0:.4f}')
    
    # Décision
    if result['reject']:
        plt.text(0.05, 0.95, "Décision: Rejeter H0", transform=plt.gca().transAxes, 
                 bbox=dict(facecolor='red', alpha=0.1))
    else:
        plt.text(0.05, 0.95, "Décision: Ne pas rejeter H0", transform=plt.gca().transAxes, 
                 bbox=dict(facecolor='green', alpha=0.1))
    
    plt.xlabel('t')
    plt.ylabel('Densité')
    plt.legend()
    plt.grid(True)
    plt.show()

In [ ]:
plot_test_student(result_bilateral_t, 'two-sided', alpha)
plot_test_student(result_left_t, 'less', alpha)
plot_test_student(result_right_t, 'greater', alpha)

# Rendu de TP

Maintenant on effectue un test sur la variance d'une loi normale avec moyenne connue.
L'objectif sera de montrer qu'on ne peut pas rejeter l'hypothèse nulle pour un échantillon généré à partir d'une loi normale de paramètres $\mu = 0$, $\sigma = 1$ et $\alpha = 0.05$. On fixe $\sigma_0 = 2$.

Pour cela, vous devrez :
1. Générer un échantillon de taille $n = 400$ à partir d'une loi normale de paramètres $\mu = 0$ et $\sigma = 1$
2. Réaliser les tests sur la variance avec $\sigma_0 = 2$ et $\alpha = 0.05$
3. Visualiser les tests réalisés
